In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [13]:
#reading csvs from raw data sets
pop2020_df = pd.read_csv("pop2020.csv")
pop2019_df=pd.read_csv("pop2019.csv", encoding="ISO-8859-1")
inventory_df = pd.read_csv("Inventory.csv")
rental_df = pd.read_csv("Rental_price.csv")

In [14]:
#cleaning popuation data sets (finding common cities from population data sets)
#selecting wanted data 
pop2020_df = pop2020_df.iloc[:, 1:4]
pop2019_df=pop2019_df.loc[:, colums]


#renaming and reordering the columns 
pop2020_df=pop2020_df.rename(columns={"Name" : "City",
                                      "2020 Pop":"2020 Population"})
pop2020_df = pop2020_df.replace({"New York City": "New York"})
colums = 'NAME', 'STNAME', 'POPESTIMATE2018', 'POPESTIMATE2019'
pop2019_df=pop2019_df.rename(columns={"NAME" : "City",
                                      "STNAME" : "State",
                                      "POPESTIMATE2018": "2018 Population",
                                      "POPESTIMATE2019" : "2019 Population"})


In [15]:
#merging population data sets

pop2020_df['City'] = pop2020_df['City'] + ' city'
population_df = pd.merge(pop2019_df, pop2020_df, on=["City", "State"], how='inner')
duplicated = population_df.loc[population_df.duplicated(subset=["City", "State"]), "City"]
pop_df = population_df.drop(duplicated.index)
pop_df.reset_index(drop=True, inplace=True)
pop_df["City"] = pop_df["City"].str.replace("city", "")

#abbrev. state names
us_state_abbrev = {
'Alabama': 'AL', 'Alaska': 'AK', 'Arizona': 'AZ', 'Arkansas': 'AR', 'California': 'CA', 'Colorado': 'CO',
'Connecticut': 'CT', 'Delaware': 'DE', 'Florida': 'FL', 'Georgia': 'GA', 'Hawaii': 'HI', 'Idaho': 'ID',
'Illinois': 'IL', 'Indiana': 'IN', 'Iowa': 'IA', 'Kansas': 'KS', 'Kentucky': 'KY', 'Louisiana': 'LA',
'Maine': 'ME', 'Maryland': 'MD', 'Massachusetts': 'MA', 'Michigan': 'MI', 'Minnesota': 'MN', 'Mississippi': 'MS',
'Missouri': 'MO', 'Montana': 'MT', 'Nebraska': 'NE', 'Nevada': 'NV', 'New Hampshire': 'NH', 'New Jersey': 'NJ',
'New Mexico': 'NM', 'New York': 'NY', 'North Carolina': 'NC', 'North Dakota': 'ND', 'Ohio': 'OH', 'Oklahoma': 'OK',
'Oregon': 'OR', 'Pennsylvania': 'PA', 'Rhode Island': 'RI', 'South Carolina': 'SC', 'South Dakota': 'SD',
'Tennessee': 'TN', 'Texas': 'TX', 'Utah': 'UT', 'Vermont': 'VT', 'Virginia': 'VA', 'Washington': 'WA',
'West Virginia': 'WV', 'Wisconsin': 'WI', 'Wyoming': 'WY'}

pop_df['State'] = pop_df['State'].map(us_state_abbrev).fillna(pop_df['State'])

#exporting data as csv file 
pop_df.to_csv("cleaned_population.csv")

In [16]:
#renaming column name and selecting wanted data 
inventory_df = inventory_df.rename(columns={"RegionName":"City"})
inventory_df.drop(inventory_df.iloc[:, 3:125], inplace = True, axis = 1) 
inventory_df.drop(inventory_df.iloc[:, 0:2], inplace = True, axis = 1) 

#renaming columns and data
inventory_df['City'] = inventory_df['City'].str.split(',').str[0]
inventory_df = inventory_df.replace({"Los Angeles-Long Beach-Anaheim": "Los Angeles",
                                "Dallas-Fort Worth":"Dallas",
                                 "Miami-Fort Lauderdale":"Miami",
                                 "Minneapolis-St Paul" : "Minneapolis",
                                 "Louisville-Jefferson County":"Louisville"})

#mering with common city list 
inventory_df = pd.merge(pop_df["City"], inventory_df, on="City", how="outer")
new_inventory_df=inventory_df.dropna(how="any")
new_inventory_df.reset_index(drop=True, inplace=True)
new_inventory_df.to_csv("cleaned_inventory.csv")
new_inventory_df

,City,2018-02-28,2018-03-31,2018-04-30,2018-05-31,2018-06-30,2018-07-31,2018-08-31,2018-09-30,2018-10-31,...,2019-11-30,2019-12-31,2020-01-31,2020-02-29,2020-03-31,2020-04-30,2020-05-31,2020-06-30,2020-07-31,2020-08-31
0,United States,238000.0,243000.0,245000.0,249500.0,255000.0,250000.0,249900.0,245000.0,244000.0,...,258000.0,260000.0,254000.0,259900.0,265000.0,267000.0,264820.0,272500.0,284000.0,285500.0
1,New York,399840.0,395000.0,400000.0,414782.0,425000.0,430000.0,430000.0,420000.0,408147.0,...,419986.0,420000.0,423000.0,423750.0,420000.0,435000.0,434950.0,445000.0,448250.0,455000.0
2,Los Angeles,625000.0,630000.0,635000.0,650000.0,652000.0,645000.0,650000.0,635000.0,635000.0,...,650000.0,655000.0,655000.0,658000.0,679000.0,665000.0,650000.0,681500.0,710000.0,725000.0
3,Chicago,217500.0,230000.0,239000.0,240000.0,249900.0,243000.0,233000.0,229000.0,222000.0,...,231250.0,230000.0,223000.0,225000.0,248000.0,260000.0,251000.0,254000.0,262000.0,270000.0
4,Dallas,255000.0,265000.0,265000.0,275000.0,280000.0,274000.0,267000.0,260000.0,257000.0,...,270000.0,275000.0,265000.0,273900.0,280000.0,285000.0,280000.0,290000.0,300000.0,300000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116,Faribault,218500.0,194900.0,220000.0,202750.0,221000.0,222318.0,236306.0,225000.0,208676.0,...,237500.0,240000.0,225450.0,226200.0,255500.0,253750.0,265000.0,267983.0,280000.0,264000.0
117,Albemarle,135000.0,143000.0,155000.0,143250.0,142750.0,139000.0,164000.0,150000.0,151500.0,...,169500.0,194000.0,157000.0,192000.0,168000.0,169000.0,175000.0,200000.0,184100.0,183000.0
118,Kerrville,215000.0,235000.0,253125.0,250000.0,266000.0,257500.0,223500.0,228000.0,263000.0,...,239750.0,256750.0,268000.0,273500.0,278500.0,275079.0,304500.0,265650.0,281063.0,298750.0
119,Shelbyville,160000.0,155500.0,160000.0,166800.0,164000.0,162500.0,171000.0,194900.0,161000.0,...,191500.0,205000.0,187495.0,174950.0,189000.0,189950.0,209150.0,187000.0,213000.0,198000.0


In [17]:
#renaming column name and selecting wanted data 
rental_df = rental_df.rename(columns={"RegionName":"City"})
rental_df.drop(rental_df.iloc[:, 2:54], inplace = True, axis = 1) 
del rental_df ["RegionID"]

#renaming columns and data
rental_df['City'] = rental_df['City'].str.split(',').str[0]
rental_df = rental_df.replace({"Los Angeles-Long Beach-Anaheim": "Los Angeles",
                                "Dallas-Fort Worth":"Dallas",
                                 "Miami-Fort Lauderdale":"Miami",
                                 "Minneapolis-St Paul" : "Minneapolis",
                                 "Louisville-Jefferson County":"Louisville"})

#mering with common city list 
rental_df = pd.merge(pop_df["City"], rental_df, on="City", how="outer")
new_rental_df=rental_df.dropna(how="any")
new_rental_df.reset_index(drop=True, inplace=True)
new_rental_df.to_csv("cleaned_rental.csv")
new_rental_df

,City,2018-04,2018-05,2018-06,2018-07,2018-08,2018-09,2018-10,2018-11,2018-12,...,2019-12,2020-01,2020-02,2020-03,2020-04,2020-05,2020-06,2020-07,2020-08,2020-09
0,United States,1604.0,1609.0,1614.0,1619.0,1624.0,1629.0,1634.0,1639.0,1644.0,...,1698.0,1700.0,1702.0,1704.0,1706.0,1707.0,1709.0,1710.0,1711.0,1712.0
1,New York,2608.0,2612.0,2617.0,2621.0,2626.0,2631.0,2636.0,2642.0,2647.0,...,2698.0,2694.0,2686.0,2678.0,2671.0,2662.0,2653.0,2644.0,2635.0,2625.0
2,Los Angeles,2405.0,2414.0,2423.0,2432.0,2440.0,2448.0,2456.0,2464.0,2472.0,...,2542.0,2544.0,2545.0,2545.0,2546.0,2546.0,2546.0,2546.0,2545.0,2545.0
3,Chicago,1713.0,1716.0,1719.0,1722.0,1725.0,1728.0,1732.0,1735.0,1739.0,...,1778.0,1778.0,1777.0,1776.0,1776.0,1775.0,1774.0,1772.0,1771.0,1770.0
4,Dallas,1481.0,1485.0,1489.0,1493.0,1498.0,1502.0,1506.0,1511.0,1515.0,...,1560.0,1562.0,1563.0,1564.0,1566.0,1567.0,1568.0,1570.0,1571.0,1572.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101,Port St. Lucie,1512.0,1520.0,1529.0,1538.0,1545.0,1552.0,1559.0,1563.0,1568.0,...,1640.0,1648.0,1656.0,1664.0,1671.0,1679.0,1687.0,1695.0,1702.0,1710.0
102,Fort Collins,1555.0,1559.0,1564.0,1569.0,1574.0,1579.0,1584.0,1589.0,1594.0,...,1636.0,1638.0,1641.0,1644.0,1646.0,1649.0,1651.0,1653.0,1656.0,1658.0
103,Boulder,1901.0,1906.0,1911.0,1916.0,1921.0,1926.0,1931.0,1936.0,1941.0,...,1991.0,1992.0,1993.0,1995.0,1996.0,1997.0,1998.0,1998.0,1999.0,2000.0
104,Greeley,1586.0,1592.0,1598.0,1604.0,1611.0,1617.0,1624.0,1629.0,1635.0,...,1680.0,1682.0,1685.0,1688.0,1690.0,1693.0,1695.0,1698.0,1700.0,1703.0
